In [7]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import folium

center_lat = 38.03169007
center_lon = -78.51091409

data = pd.read_csv('final_data.csv')
locations = pd.read_csv('locations.csv')

data = data.dropna()

# print(data.columns)
# print(locations.columns)

locations = locations[['location_name', 'latitude', 'longitude']]

locations['longitude'] = locations['longitude'].astype(float).apply(lambda x: -x)




# Merge the data and locations DataFrames
data = data.merge(locations, on='location_name')

data = data.where(data.gateway_id=='eui-9cd47dfffe9e0450')

data = data.dropna()

data = data.reset_index(drop=True)

# print(data.iloc[128])
# return

# data.to_csv('final_data_1.csv')

# take the median spreading factor for each location_name
median_data = data.groupby('location_name').agg({'latitude':'median', 'longitude':'median', 'spreading_factor':'median'}).reset_index()

# Create a map centered around Charlottesville with adjusted marker sizes based on spreading_factor
charlottesville_map_adjusted = folium.Map(location=[center_lat, center_lon], zoom_start=15)

# Scale factor for the circle size
scale_factor = 1

# Define a function to choose color based on spreading factor
def choose_color(sf):
    if sf == 7:
        return '#ffc100'
    elif sf == 8:
        return '#ff9a00'
    elif sf == 9:
        return '#ff7400'
    elif sf == 9.5:
        return '#ff4d00'
    elif sf == 10:
        return '#ff0000'
    else:
        return 'gray'  # Default color for other values

# Loop through each data point and create a CircleMarker with the appropriate color
for lat, lon, sf in zip(median_data['latitude'], median_data['longitude'], median_data['spreading_factor']):
    folium.CircleMarker(
        location=[lat, lon],
        radius=10,  # Fixed size, or adjust as necessary
        color=choose_color(sf),
        fill=True,
        fill_color=choose_color(sf),
        fill_Opacity = 1
    ).add_to(charlottesville_map_adjusted)


folium.CircleMarker(
    location=[center_lat, center_lon],
    radius=5,
    color='blue',
    fill=True,
    fill_color='blue'
).add_to(charlottesville_map_adjusted
)

# Add a circle with a 0.804 km radius to the map centered at the given coordinates
folium.Circle(
    location=[center_lat, center_lon],
    radius=804,  # radius in meters (0.804 kilometers)
    color='black',
    fill=True,
    fill_color='black',
    fill_opacity=0.2,
    weight=1.5  # line thickness
).add_to(charlottesville_map_adjusted)

# Add a circle with a 1.608 km radius to the map centered at the given coordinates
folium.Circle(
    location=[center_lat, center_lon],
    radius=1608,  # radius in meters (0.804 kilometers)
    color='black',
    fill=True,
    fill_color='black',
    fill_opacity=0.2,
    weight=1.5  # line thickness
).add_to(charlottesville_map_adjusted)


# Creating a legend with color boxes
legend_html = '''
<div style="position: fixed; 
     top: 50px; right: 50px; width: 150px; height: 140px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: white;
     "&nbsp; <b>Spreading Factor Legend</b> <br>
     &nbsp; <i style="background-color:#ffc100; width:10px; height:10px; display:inline-block; margin-right: 5px;"></i>7<br>
     &nbsp; <i style="background-color:#ff9a00; width:10px; height:10px; display:inline-block; margin-right: 5px;"></i>8<br>
     &nbsp; <i style="background-color:#ff7400; width:10px; height:10px; display:inline-block; margin-right: 5px;"></i>9<br>
     &nbsp; <i style="background-color:#ff4d00; width:10px; height:10px; display:inline-block; margin-right: 5px;"></i>9.5<br>
     &nbsp; <i style="background-color:#FF0000; width:10px; height:10px; display:inline-block; margin-right: 5px;"></i>10<br>
</div>
'''

# Add the legend to the map
charlottesville_map_adjusted.get_root().html.add_child(folium.Element(legend_html))


# Save the updated map
charlottesville_map_adjusted.save("Charlottesville_Transmission_Map_Median.html")
charlottesville_map_adjusted

In [6]:
import numpy as np

# take the variance of spreading factor for each location_name
variance_data = data.groupby('location_name').agg({'latitude':'median', 'longitude':'median', 'spreading_factor':'var'}).reset_index()
variance_data.rename(columns={'spreading_factor': 'sf_variance'}, inplace=True)

charlottesville_map_adjusted1 = folium.Map(location=[center_lat, center_lon], zoom_start=15)

import matplotlib.colors as mcolors

# Create a color map
cmap = mcolors.LinearSegmentedColormap.from_list("mycmap", ["#ffc100", "#ff0000"])

# Normalize the variance for color mapping
sf_max = variance_data['sf_variance'].max()
sf_min = variance_data['sf_variance'].min()
norm = mcolors.Normalize(vmin=sf_min, vmax=sf_max)

for lat, lon, sf_var in zip(variance_data['latitude'], variance_data['longitude'], variance_data['sf_variance']):
    # Get color from colormap
    color = mcolors.to_hex(cmap(norm(sf_var)))
    
    folium.CircleMarker(
        location=[lat, lon],
        radius=10,  # Fixed size, or adjust as necessary
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=1
    ).add_to(charlottesville_map_adjusted1)

folium.CircleMarker(
    location=[center_lat, center_lon],
    radius=5,
    color='blue',
    fill=True,
    fill_color='blue'
).add_to(charlottesville_map_adjusted1)

# Add a circle with a 0.804 km radius to the map centered at the given coordinates
folium.Circle(
    location=[center_lat, center_lon],
    radius=804,  # radius in meters (0.804 kilometers)
    color='black',
    fill=True,
    fill_color='black',
    fill_opacity=0.2,
    weight=1.5  # line thickness
).add_to(charlottesville_map_adjusted1)

# Add a circle with a 1.608 km radius to the map centered at the given coordinates
folium.Circle(
    location=[center_lat, center_lon],
    radius=1608,  # radius in meters (0.804 kilometers)
    color='black',
    fill=True,
    fill_color='black',
    fill_opacity=0.2,
    weight=1.5  # line thickness
).add_to(charlottesville_map_adjusted1)

from branca.colormap import LinearColormap

# Define a color bar
color_bar = LinearColormap(colors=["#ffc100", "#ff0000"], vmin=sf_min, vmax=sf_max, caption='Spreading Factor Variance')

charlottesville_map_adjusted1.add_child(color_bar)

charlottesville_map_adjusted1.save("Charlottesville_Transmission_Map_Variance.html")
charlottesville_map_adjusted1